Grupo
- Beatriz (2023007113)
- Laviny (2023012893)
- Caio (2022012333)

#### Etapa 1
- Nosso conjunto de dados é o Seeds
- Foi sugerido retirar 1 ou 2 atributos para o trabalho

Realizar a importação dos dados a partir do CSV e definir os nomes das colunas 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Nomes das colunas
column_names = [
    "Área",
    "Perímetro",
    "Compacidade",
    "Comprimento_grão",
    "Largura_grão",
    "Coeficiente_assimetria",
    "Comprimento_sulco",
    "Classe"
]

# Leitura do CSV original
csv_file = 'seeds_dataset.csv'
df_original = pd.read_csv(csv_file, header=None, delim_whitespace=True, names=column_names)

Imprimir informações básicas sobre o dataset. Essa etapa é importante para compreender a estrutura dos dados antes de iniciar as modificações e análises posteriores

In [ ]:
df_original.info()

Uma verificação importante a ser feita é ver como está a distribuição das classes do dataset

A coluna "Classe" representa os rótulos do conjunto de dados. Como queremos descobrir padrões e estruturas sem usar a informação de rótulo, ou seja, a abordagem é não supervisionada. Removendo a coluna "Classe", garantimos que o algoritmo não seja influenciado por essa informação e possa identificar os grupos baseados somente nas características dos dados.



In [ ]:
df_original['Classe'].value_counts()

Nesta etapa, removemos a coluna "Perímetro" (além de "Classe") para criar um conjunto modificado dos dados

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


X = df_original.drop(columns=["Perímetro", "Classe"])
y = df_original["Classe"]


---------------------

#### Etapa 2
- Aplicar uma técnica de classificação: KNN
- Aplicar uma técnica de agrupamento: K-médias

Em seguida, separamos as features (X) dos rótulos (y) e dividimos os dados em conjuntos de treinamento e teste. O StandardScaler é utilizado para normalizar os dados, etapa essencial para o funcionamento adequado do algoritmo KNN que será aplicado para a tarefa de classificação


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalização dos dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#Classificação com KNN (K-Nearest Neighbors)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)       

y_pred = knn.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do KNN: {accuracy:.4f}")

Aqui, aplicamos o modelo KNN utilizando os dados originais. O modelo é treinado com os dados de treinamento e avaliado em termos de acurácia a partir do conjunto de teste

In [ ]:
X_full = df_original.drop(columns=["Classe"])
y_full = df_original["Classe"]

X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

y_pred = knn.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do KNN: {accuracy:.4f}")

Como obtemos a mesma acurácia para ambos, isso pode significar que o atributo que escolhemos para remover (Perímetro) é irrelevante, já que o desempenho se mostrou o mesmo

Se realizarmos o teste retirando outro atributo, a acurácia muda. Por exemplo, se retirarmos a Compacidade ao invés do Perímetro

In [ ]:
X_teste = df_original.drop(columns=["Compacidade", "Classe"])
y_teste = df_original["Classe"]

X_train, X_test, y_train, y_test = train_test_split(X_teste, y_teste, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

y_pred = knn.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do KNN: {accuracy:.4f}")

Quando o conjunto modificado apresenta acurácia melhor do que o conjunto original, isso pode indicar que o atributo removido estava introduzindo ruído ou informações redundantes. Ao eliminar essas variáveis que não contribuem com a capacidade do modelo de aprender padrões relevantes, o modelo consegue focar melhor na informação útil, resultando em um melhor desempenho na classificação

------------------------------

#### Técnica de agrupamento: K-médias


Para avaliar o desempenho do modelo de agrupamento KMeans utilizamos o 'silhouette_score'. Onde os valores variam entre -1 e 1. 
- próximo de 1: pontos bem agrupados
- igual a 0: os pontos estão na borda entre os dois clusters
- próximo de -1: pontos mal agrupados

Nesta parte, removemos as colunas "Perímetro" e "Classe" para evitar a interferência dos rótulos (que não devem ser usados em técnicas não supervisionadas). Em seguida, os dados são normalizados e o algoritmo K‑Médias é aplicado para formar dois clusters (o número de clusters pode ser ajustado conforme necessário). O Silhouette Score é calculado para avaliar a qualidade dos agrupamentos e, através de PCA, os clusters são visualizados em um gráfico de dispersão.

In [ ]:
from sklearn.cluster import KMeans                       
from sklearn.preprocessing import StandardScaler        
from sklearn.decomposition import PCA                   
from sklearn.metrics import silhouette_score             
import matplotlib.pyplot as plt                          

X = df_original.drop(columns=["Perímetro", "Classe"])

# Escalonamento dos dados, dados com média 0 e desvio padrão 1
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Definindo o número de clusters (aqui usamos 2 como exemplo)
n_clusters = 2

# Aplicando KMeans
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
labels = kmeans.fit_predict(X_scaled)

# Avaliação do agrupamento usando o Silhouette Score
sil = silhouette_score(X_scaled, labels)
print(f"Silhouette Score: {sil:.4f}")

# Visualização dos clusters usando PCA para redução de dimensionalidade
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)

plt.figure(figsize=(6, 4))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap="viridis", alpha=0.7)
plt.title("K‑médias no dataset modificado")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.colorbar(scatter, label="Cluster")
plt.tight_layout()
plt.show()

Neste último trecho, mantemos todas as colunas dos dados (exceto "Classe", que é removida para evitar vazamento de informação) e aplicamos o mesmo procedimento de escalonamento, clusterização e avaliação dos grupos formados. Essa etapa possibilita a comparação entre o agrupamento obtido a partir do conjunto original e o conjunto modificado, permitindo uma análise comparativa dos resultados.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

X_full = df_original.drop(columns=["Classe"])

scaler = StandardScaler()
X_scaled_full = scaler.fit_transform(X_full)

n_clusters = 2

kmeans = KMeans(n_clusters=n_clusters, random_state=42)
labels = kmeans.fit_predict(X_scaled_full)

sil = silhouette_score(X_scaled_full, labels)
print(f"Silhouette Score: {sil:.4f}")

pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled_full)

plt.figure(figsize=(6, 4))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap="viridis", alpha=0.7)
plt.title("K‑médias no dataset original")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.colorbar(scatter, label="Cluster")
plt.tight_layout()
plt.show()

Quando o Silhouette Score do conjunto original é maior do que o do conjunto modificado, isso sugere que os clusters formados com as features originais estão mais  separados. Os atributos removidos podem ter contribuído para uma melhor definição dos grupos, e sua retirada gerou em uma estrutura de agrupamento menos distinta 

Se refizermos o teste mas agora retirando a Compacidade

In [ ]:

X = df_original.drop(columns=["Compacidade", "Classe"])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

n_clusters = 2

kmeans = KMeans(n_clusters=n_clusters, random_state=42)
labels = kmeans.fit_predict(X_scaled)


sil = silhouette_score(X_scaled, labels)
print(f"Silhouette Score: {sil:.4f}")

pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)

plt.figure(figsize=(6, 4))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap="viridis", alpha=0.7)
plt.title("K‑médias no dataset modificado")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.colorbar(scatter, label="Cluster")
plt.tight_layout()
plt.show()

Aconteceu da mesma forma que na classificação, o Silhouette Score ficou melhor nesse conjunto modificado, indicando que a remoção de Compacidade melhorou a estrutura dos grupos, deixando melhor separados

--------------------------

In [ ]:
# Faz o Jupyter exibir tudo sem cortar
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Etapa 3


Classificação usando KNN

Explicação do Relatório de Classificação:

- precision: O quanto você pode confiar nas previsões do modelo. "Se o modelo disse que é, qual a chance de realmente ser?"

- recall: Se o modelo consegue encontrar todos os casos positivos. "De todos os que realmente são, quantos o modelo achou?"

- f1-score: Média entre precision e recall

- support: Número de amostras reais de cada classe


Parâmetros usados na classificação com KNN: 

- 'n_neighbors=5': o número de vizinhos usados para classificar cada ponto. Quanto menor o valor, mais sensível o modelo fica ao ruído.
- 'weights': peso dos vizinhos, podem ser 'uniform' ou 'distance'.
- 'metric': métrica da distância.

Teste sem Perímetro

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

X_teste = df_original.drop(columns=["Perímetro", "Classe"])
y_teste = df_original["Classe"]

X_train, X_test, y_train, y_test = train_test_split(X_teste, y_teste, test_size=0.3, random_state=42)

# Normalização dos dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Loop por todas as combinações de hiperparâmetros
for n in [3, 5, 7]:
    for w in ['uniform', 'distance']:
        for m in ['euclidean', 'manhattan']:
            print(f"\nTestando com n_neighbors={n}, weights='{w}', metric='{m}'")

            # Instancia e treina o modelo
            knn = KNeighborsClassifier(n_neighbors=n, weights=w, metric=m)
            knn.fit(X_train_scaled, y_train)
            y_pred = knn.predict(X_test_scaled)

            acc = accuracy_score(y_test, y_pred)
            print(f"Acurácia: {acc:.4f}")
            print("Relatório de Classificação:")
            print(classification_report(y_test, y_pred))
            print("Matriz de Confusão:")
            print(confusion_matrix(y_test, y_pred))


Com n_neighbors = 7 a acurácia cai em dois exemplos de testes diferentes, enquanto que os outros teste a acurácia se mantém = 90.

#### Dataset sem a Compacidade

In [ ]:

X = df_original.drop(columns=["Compacidade", "Classe"])
y = df_original["Classe"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalização dos dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Loop por todas as combinações de hiperparâmetros
for n in [3, 5, 7]:
    for w in ['uniform', 'distance']:
        for m in ['euclidean', 'manhattan']:
            print(f"\nTestando com n_neighbors={n}, weights='{w}', metric='{m}'")

            # Instancia e treina o modelo
            knn = KNeighborsClassifier(n_neighbors=n, weights=w, metric=m)
            knn.fit(X_train_scaled, y_train)
            y_pred = knn.predict(X_test_scaled)

            acc = accuracy_score(y_test, y_pred)
            print(f"Acurácia: {acc:.4f}")
            print("Relatório de Classificação:")
            print(classification_report(y_test, y_pred))
            print("Matriz de Confusão:")
            print(confusion_matrix(y_test, y_pred))


Em geral os testes resultaram acurácias altas. A mais baixa com a combinação de n_neighbors=7, weights='uniform', metric='euclidean'.

Dataset original

In [ ]:

X = df_original.drop(columns=["Classe"])
y = df_original["Classe"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalização dos dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Loop por todas as combinações de hiperparâmetros
for n in [3, 5, 7]:
    for w in ['uniform', 'distance']:
        for m in ['euclidean', 'manhattan']:
            print(f"\nTestando com n_neighbors={n}, weights='{w}', metric='{m}'")

            # Instancia e treina o modelo
            knn = KNeighborsClassifier(n_neighbors=n, weights=w, metric=m)
            knn.fit(X_train_scaled, y_train)
            y_pred = knn.predict(X_test_scaled)

            acc = accuracy_score(y_test, y_pred)
            print(f"Acurácia: {acc:.4f}")
            print("Relatório de Classificação:")
            print(classification_report(y_test, y_pred))
            print("Matriz de Confusão:")
            print(confusion_matrix(y_test, y_pred))


Houve 4 testes com acurácia = 0.89

Apenas 2 testes com acurácia > 0.90

Houve 6 testes com acurácia = 0.90

--------------------------

#### Teste com K-Means

Parâmetros da técnica K-Means:
- n_clusters: Quantidade de clusters.
- init: Método de inicialização dos centróides.
- n_init: Número de vezes que será executado com diferentes inicializações.
- max_iter: Número de iterações por execução.
- random_state: Semente para reprodutibilidade.
- algorithm: Algoritmo que será usado.

Teste sem Perímetro

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

X = df_original.drop(columns=["Perímetro", "Classe"])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Lista com configurações variadas de parâmetros
parametros_kmeans = [
    {"n_clusters": 3, "init": "k-means++", "n_init": 10, "max_iter": 300, "algorithm": "lloyd"},
    {"n_clusters": 4, "init": "random", "n_init": 20, "max_iter": 500, "algorithm": "elkan"},
    {"n_clusters": 5, "init": "k-means++", "n_init": 20, "max_iter": 500, "algorithm": "elkan"},
    {"n_clusters": 6, "init": "random", "n_init": 20, "max_iter": 500, "algorithm": "elkan"},
    {"n_clusters": 7, "init": "k-means++", "n_init": 10, "max_iter": 100, "algorithm": "lloyd"},
    {"n_clusters": 8, "init": "random", "n_init": 15, "max_iter": 200, "algorithm": "elkan"},
]

# Redução de dimensionalidade para visualização
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)

# Gráficos
plt.figure(figsize=(15, 12))

for i, params in enumerate(parametros_kmeans, 1):
    kmeans = KMeans(
        n_clusters=params["n_clusters"],
        init=params["init"],
        n_init=params["n_init"],
        max_iter=params["max_iter"],
        algorithm=params["algorithm"],
        random_state=42
    )
    labels = kmeans.fit_predict(X_scaled)
    sil = silhouette_score(X_scaled, labels)

    plt.subplot(3, 2, i)
    scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap="tab10", alpha=0.7)
    plt.title(f"Clusters: {params['n_clusters']}, Init: {params['init']}\n"
              f"Iter: {params['max_iter']}, Algo: {params['algorithm']}")
    plt.xlabel("PC1")
    plt.ylabel("PC2")

    # Mostrar Silhouette Score
    plt.text(0.05, 0.95, f"Silhouette: {sil:.4f}", transform=plt.gca().transAxes,
             fontsize=10, verticalalignment='top', bbox=dict(boxstyle="round", facecolor="white", alpha=0.5))

plt.tight_layout()
plt.show()


Teste em dataset original

In [ ]:
X_full = df_original.drop(columns=["Classe"])

scaler = StandardScaler()
X_scaled_full = scaler.fit_transform(X_full)

# Lista com configurações variadas de parâmetros
parametros_kmeans = [
    {"n_clusters": 2, "init": "k-means++", "n_init": 10, "max_iter": 300, "algorithm": "lloyd"},
    {"n_clusters": 3, "init": "random", "n_init": 20, "max_iter": 500, "algorithm": "elkan"},
    {"n_clusters": 4, "init": "k-means++", "n_init": 20, "max_iter": 500, "algorithm": "elkan"},
    {"n_clusters": 5, "init": "random", "n_init": 20, "max_iter": 500, "algorithm": "elkan"},
    {"n_clusters": 6, "init": "k-means++", "n_init": 10, "max_iter": 100, "algorithm": "lloyd"},
    {"n_clusters": 7, "init": "random", "n_init": 15, "max_iter": 200, "algorithm": "elkan"},
]

# Redução de dimensionalidade para visualização
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)

# Gráficos
plt.figure(figsize=(15, 12))

for i, params in enumerate(parametros_kmeans, 1):
    kmeans = KMeans(
        n_clusters=params["n_clusters"],
        init=params["init"],
        n_init=params["n_init"],
        max_iter=params["max_iter"],
        algorithm=params["algorithm"],
        random_state=42
    )
    labels = kmeans.fit_predict(X_scaled)
    sil = silhouette_score(X_scaled, labels)

    plt.subplot(3, 2, i)
    scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap="tab10", alpha=0.7)
    plt.title(f"Clusters: {params['n_clusters']}, Init: {params['init']}\n"
              f"Iter: {params['max_iter']}, Algo: {params['algorithm']}")
    plt.xlabel("PC1")
    plt.ylabel("PC2")

    # Mostrar Silhouette Score
    plt.text(0.05, 0.95, f"Silhouette: {sil:.4f}", transform=plt.gca().transAxes,
             fontsize=10, verticalalignment='top', bbox=dict(boxstyle="round", facecolor="white", alpha=0.5))

plt.tight_layout()
plt.show()


Teste sem Compacidade

In [ ]:
X = df_original.drop(columns=["Compacidade", "Classe"])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Lista com configurações variadas de parâmetros
parametros_kmeans = [
    {"n_clusters": 3, "init": "k-means++", "n_init": 10, "max_iter": 300, "algorithm": "lloyd"},
    {"n_clusters": 4, "init": "random", "n_init": 20, "max_iter": 500, "algorithm": "elkan"},
    {"n_clusters": 5, "init": "k-means++", "n_init": 20, "max_iter": 500, "algorithm": "elkan"},
    {"n_clusters": 6, "init": "random", "n_init": 20, "max_iter": 500, "algorithm": "elkan"},
    {"n_clusters": 7, "init": "k-means++", "n_init": 10, "max_iter": 100, "algorithm": "lloyd"},
    {"n_clusters": 8, "init": "random", "n_init": 15, "max_iter": 200, "algorithm": "elkan"},
]

# Redução de dimensionalidade para visualização
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)

# Gráficos
plt.figure(figsize=(15, 12))

for i, params in enumerate(parametros_kmeans, 1):
    kmeans = KMeans(
        n_clusters=params["n_clusters"],
        init=params["init"],
        n_init=params["n_init"],
        max_iter=params["max_iter"],
        algorithm=params["algorithm"],
        random_state=42
    )
    labels = kmeans.fit_predict(X_scaled)
    sil = silhouette_score(X_scaled, labels)

    plt.subplot(3, 2, i)
    scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap="tab10", alpha=0.7)
    plt.title(f"Clusters: {params['n_clusters']}, Init: {params['init']}\n"
              f"Iter: {params['max_iter']}, Algo: {params['algorithm']}")
    plt.xlabel("PC1")
    plt.ylabel("PC2")

    # Mostrar Silhouette Score
    plt.text(0.05, 0.95, f"Silhouette: {sil:.4f}", transform=plt.gca().transAxes,
             fontsize=10, verticalalignment='top', bbox=dict(boxstyle="round", facecolor="white", alpha=0.5))

plt.tight_layout()
plt.show()

Dentre os três exemplos o melhor modo de fazer a análise foi sem a Compacidade, pois os pontos foram melhor agrupados.